# Disjoint-domain network

### Ethan Blackwood
### October 23, 2020

**Goal**: Train and analyze the network in Rogers/McClelland 2008 with 4 disjoint domains (Figures R3-R5), which learns to extract the feature of being more or less similar to other items in the same domain, across the 4 domains which have no items, contexts or attributes in common.

In [1]:
import numpy as np
from datetime import datetime as dt

import torch

import disjoint_domain as dd
import ddnet

device, torchfp = dd.init_torch()
if device.type == 'cuda':
    print('Using CUDA')
else:
    print('Not using CUDA')

ctx_per_domain, n_domains, n_items, n_ctx, attrs_per_context = dd.get_net_defaults()

Using CUDA


Common training procedure:

In [2]:
def train_n_dd_nets(n=36, run_type='dd', snap_report_freq=100, **net_params):

    num_epochs = 4000
    batch_size = 16
    snap_epochs = dd.calc_snap_epochs(snap_report_freq, 'lin', num_epochs)

    all_snaps = {'item': [], 'context': []}
    all_reports = {'loss': [], 'accuracy': [], 'etg': []}

    for i in range(n):
        net = ddnet.DisjointDomainNet(ctx_per_domain, attrs_per_context, n_domains,
                                      device=device, torchfp=torchfp, **net_params)
        optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
        #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3000, gamma=0.95)
        scheduler = None

        print(f'Training Iteration {i+1}')
        print('---------------------')
        snaps, snap_epochs, reports, report_epochs = net.do_training(
            optimizer, num_epochs=num_epochs, batch_size=batch_size, report_freq=snap_report_freq,
            snap_freq=snap_report_freq, scheduler=scheduler, do_holdout_testing=True)

        for snap_type in ['item', 'context']:
            all_snaps[snap_type].append(snaps[snap_type])

        for report_type in ['loss', 'accuracy', 'etg']:
            all_reports[report_type].append(reports[report_type])

        print('')

    np.savez(f'data/{run_type}_res_{dt.now():%Y-%m-%d_%H-%M-%S}',
             snapshots=all_snaps, snap_freq=snap_report_freq, snap_epochs=snap_epochs,
             reports=all_reports, report_freq=snap_report_freq)

Original network, with separate item & context streams, with hold-out

In [ ]:
train_n_dd_nets()

Training Iteration 1
---------------------
Holding out item: D4*
Holding out context: A1
Epoch    0 end: loss = 215.146, acc = 0.558, epochs to gen. = >2000
Epoch  100 end: loss =   7.328, acc = 0.984, epochs to gen. = 990  
Epoch  200 end: loss =   4.430, acc = 0.991, epochs to gen. = 43   
Epoch  300 end: loss =   4.207, acc = 0.991, epochs to gen. = 42   
Epoch  400 end: loss =   4.010, acc = 0.992, epochs to gen. = 29   
Epoch  500 end: loss =   3.842, acc = 0.992, epochs to gen. = 25   
Epoch  600 end: loss =   3.681, acc = 0.992, epochs to gen. = 24   
Epoch  700 end: loss =   3.531, acc = 0.993, epochs to gen. = 32   
Epoch  800 end: loss =   3.381, acc = 0.993, epochs to gen. = 70   
Epoch  900 end: loss =   3.211, acc = 0.993, epochs to gen. = 23   
Epoch 1000 end: loss =   3.084, acc = 0.994, epochs to gen. = 31   
Epoch 1100 end: loss =   2.959, acc = 0.994, epochs to gen. = 49   
Epoch 1200 end: loss =   2.817, acc = 0.994, epochs to gen. = 17   


Network with merged representation layer (items & contexts all go to all rep units)

In [ ]:
train_n_dd_nets(run_type='merged_dd', merged=True)